In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import pandas as pd
pd.set_option("display.max_columns",500)
pd.set_option('display.max_rows', None)
# 数据读取
file_1 = pd.read_excel('附件1.xlsx')
file_2 = pd.read_csv('附件2.csv')
file_3 = pd.read_excel('附件3.xlsx')

# 数据合并
file_1.columns = ['TICKER_SYMBOL','所属行业']
data = pd.merge(file_1,file_2,how='right',on='TICKER_SYMBOL')
# 行业提取
columns_list = []
for col in data['所属行业'].value_counts().index:
    columns_list.append(col)
# 分行业进行数据预处理
for i in columns_list:
    data_one = data[data['所属行业']==i]
    print(i,data_one.shape)
    # 将标签列提取出
    flag = data_one.pop('FLAG')
    # 清除缺失值大于50%的列
    for j in data_one.columns:
        if ((data_one[j].isnull().sum() / data_one.shape[0]) >= 0.5):
            del data_one[j]
    # 缺失值20%-50%的用均值填充
    for column in data_one.columns[(data_one.isnull().sum()/data_one.shape[0]) > 0.2]:
        mean_val = data_one[column].mean()
        data_one[column].fillna(mean_val, inplace=True)
    # 使用随机森林填充
    while True:
        try:
            # 找出缺失值最小的列名
            col_name = data_one.isnull().sum()[data_one.isnull().sum()>0].index[data_one.isnull().sum()[data_one.isnull().sum()>0].argmin()]
            # 创建一个新的数据集
            RF_fill = data_one.copy()
            # 筛选出有用的指标
            RF_fill = RF_fill.iloc[:,11:]
            # 切分训练集和预测集
            RF_train = RF_fill[~RF_fill[col_name].isnull()]
            RF_train_Y = RF_train.pop(col_name)
            RF_train_X = RF_train

            RF_pre = RF_fill[RF_fill[col_name].isnull()]
            RF_pre_Y = RF_pre.pop(col_name)
            RF_pre_X = RF_pre
            # 填充X
            RF_train_X =RF_train_X.fillna(0)
            RF_pre_X = RF_pre_X.fillna(0)
            # 训练
            from sklearn.ensemble import RandomForestRegressor

            clf = RandomForestRegressor(random_state=1)
            clf.fit(RF_train_X, RF_train_Y)
            pre = clf.predict(RF_pre_X)
            # 填充
            data_one.loc[RF_pre_Y.index,col_name] = pre
            print('已填充',col_name)
        except:
            break  # 填充完毕后跳出循环
    # 最大最小归一化
    from sklearn.preprocessing import MinMaxScaler

    scaler = MinMaxScaler(feature_range=(0,1))
    data_one.iloc[:,11:-1] = scaler.fit_transform(data_one.iloc[:,11:-1])   # 排除前面的无意义字段
    # 合并标签列
    data_one['FLAG'] = flag
    
    # 保存数据
    data_one.to_csv('./清理后的数据/{}.csv'.format(i))
    print("{}完成".format(i))
    print('------------------------------------------------')